# Reinforcement Learning Project

## Setup

To be able to run this notebook properly please make sure to install the pettingzoo package and dependencies. This can be done by running the following command

`pip install pettingzoo[mpe]`

### Imports

In [ ]:
from pettingzoo.mpe import simple_world_comm_v2
import random
import numpy as np

### Environment Initialisation

In [ ]:
MAX_CYCLES = 250
NUM_OF_EPISODES = 10


env = simple_world_comm_v2.env(num_good=2, num_adversaries=4, num_obstacles=1,
                num_food=2, max_cycles=MAX_CYCLES, num_forests=2, continuous_actions=False)
env.reset()

### Policy Function

In [ ]:
def random_policy(actions):
    return random.randint(0, actions-1)

### Running the environment

The `env.render(mode='human')` call will pop open a new window that shows the environment at each time step.

On my machine at least this window can only be closed while the cell is running but then freezes and is unable to be closed afterwards. In these cases restarting the kernel closed the window and any others which may have been opened due to running the cell multiple times.

Eventually running the cell enough times without restarting the kernal will cause the render call to throw an exception and not run. In this case just restart the kernal and it will begin working again.

In [ ]:
print(f"Agents: {env.agents}")
print()
agent_mapping = {k: v for v, k in enumerate(env.agents)}
reward_array = np.zeros((NUM_OF_EPISODES,len(env.agents)))
cumulative_reward = np.zeros(len(env.agents))

for episode in range(NUM_OF_EPISODES):
    env.reset()
    for agent in env.agent_iter():
        observation, reward, done, info = env.last()
        cumulative_reward[agent_mapping[agent]] += reward
        
        
    #     Various print statements which can help see what is returned at each step.
    #     Commented out due to how verbose they are. Try toggling them on one at a time.
#         print(f"Current Agent: {env.agent_selection}")
    #     print(f"Obs: {observation}")
#         print(f"Rew: {reward}")      
    #     print(f"Done: {done}")
    #     print(f"Info: {info}")

    #     Renders the environment for each step in a seperate window.
        env.render(mode='human')

    #     Steps the environment forward.
        if done:
            env.step(None)
            reward_array[episode,agent_mapping[agent]] = cumulative_reward[agent_mapping[agent]]
        else:
            env.step(random_policy(env.action_space(agent).n))

### Print Reward Array

In [ ]:
print(reward_array)